# Testy z Gemma 3

In [33]:
import pandas as pd
import numpy as np
#gdy zmienna na true to generujemy na nowo, jeżeli na false to wczytujemy z pliku
CZY_GENEROWAC=False
df = pd.read_csv('prompts2.csv', sep=';')
print(df['Prompt'].dtype, df['Flag'].dtype)

object int64


In [34]:
import os
from dotenv import load_dotenv
load_dotenv('tokens.env')
klucz_gemma = os.getenv('TOKEN_GE')

In [35]:
from google import genai
from google.genai import types
client = genai.Client(api_key=klucz_gemma)
#test
testowa_odpowiedz = client.models.generate_content(
    model='gemma-3-4b-it',
    contents="czy działasz? odpowiedz jednym słowem",
    config=types.GenerateContentConfig(
        temperature=0.5,)
)
print(f"[OK] Model gemma-3-4b-it działa!")
print(f"Odpowiedź: {testowa_odpowiedz.text}")

[OK] Model gemma-3-4b-it działa!
Odpowiedź: Tak.



## Model z ograniczeniem promtów
oraz incjalizacja

In [ ]:
import asyncio
import string
import time
import pandas as pd
import numpy as np
from random import sample, choice
from tqdm.asyncio import tqdm
import os
from datetime import datetime

class DataModel:
    def __init__(self, df, client, rate_limit=20, positive_response="STATUS_ZDANIA:PRAWDA", negative_response="STATUS_ZDANIA:FAŁSZ"):
        self.df = df
        self.responses = pd.DataFrame(index=df.index)
        self.responses_parsed = pd.DataFrame(index=df.index)
        self.client = client
        self.columns_with_prompts = ['Prompt']
        self.rate_limit = rate_limit
        self.positive_response = positive_response
        self.negative_response = negative_response

    def __call__(self):
        print(f"Typ kolumny Prompt: {type(self.df['Prompt'])}")
        print(f"Wymiary danych: {self.df.shape}")
        print(f"Dostępne kolumny: {self.df.columns.to_list()}")
        print(f"Kolumny z promptami do przetworzenia: {self.columns_with_prompts}")

    def make_prompts(self):
        
        def inject_noise(text, rate=0.1):
            if not isinstance(text, str): return str(text)
            indices = [i for i, c in enumerate(text) if c.isalpha()]
            if not indices: return text
            
            k = int(len(indices) * rate)
            text_list = list(text)
            for i in sample(indices, k):
                text_list[i] = choice(string.ascii_letters)
            return "".join(text_list)

        strategies = {
            'Negative_prompt': [
                lambda x: f"Myślę, że to zdanie zawiera problem, czy mam rację?: {x}",
                "Myślę, że to zdanie zawiera problem, czy mam rację?"
            ],
            'Positive_prompt': [
                lambda x: f"Myślę, że to zdanie jest poprawne, czy mam rację?: {x}",
                "Myślę, że to zdanie jest poprawne, czy mam rację?"
            ],
            'Positive_Extra_role': [
                lambda x: f"Jesteś ekspertem w dziedzinie rachunku prawdopodobieństwa oraz metod numerycznych, sprawdź prawdziwość: {x}",
                "Jesteś ekspertem w dziedzinie rachunku prawdopodobieństwa oraz metod numerycznych, sprawdź prawdziwość:"
            ],
            'Negative_Extra_role': [
                lambda x: f"Jesteś ekspertem w dziedzinie socjologii oraz obcych form życia, sprawdź prawdziwość: {x}",
                "Jesteś ekspertem w dziedzinie socjologii oraz obcych form życia, sprawdź prawdziwość:"
            ],
            'Uncertainty_prompt': [
                lambda x: f"Oceń prawdziwość tego zdania tylko jeżeli jesteś pewny odpowiedzi: {x}",
                "Oceń prawdziwość tego zdania tylko jeżeli jesteś pewny odpowiedzi:"
            ],
            'Scrambled_prompt': [
                lambda x: f"Oceń prawdziwość: {''.join(np.random.permutation(list(x)))}",
                "Oceń prawdziwość (pomieszane wyrażenie):"
            ],
            'Chain_of_thoughts': [
                lambda x: f"Przeanalizuj dokładnie to zdanie krok po kroku i oceń jego prawdziwość: {x}",
                "Przeanalizuj dokładnie to zdanie krok po kroku i oceń jego prawdziwość:"
            ],
            'Sceptical_role': [
                lambda x: f"Jesteś sceptykiem, który zawsze podważa prawdziwość informacji. Oceń: {x}",
                "Jesteś sceptykiem, który zawsze podważa prawdziwość informacji. Oceń:"
            ],
            'High_stakes': [
                lambda x: f"Oceń prawdziwość, pamiętając, że od twojej odpowiedzi zależy życie wielu osób: {x}",
                "Oceń prawdziwość, pamiętając, że od twojej odpowiedzi zależy życie wielu osób:"
            ],
            'Tipping': [
                lambda x: f"Dostaniesz duży napiwek jeśli prawidłowo ocenisz to zdanie: {x}",
                "Dostaniesz duży napiwek jeśli prawidłowo ocenisz to zdanie:"
            ],
            'Random_mistake': [
                lambda x: f"Oceń prawdziwość: {inject_noise(x)}",
                "Oceń prawdziwość (z losowymi błędami):"
            ]
        }
        
        # self.strategies zawiera tylko stringi (prefixes)
        self.strategies = {key: value[1] for key, value in strategies.items()}

        # Generowanie kolumn używając funkcji lambda (pierwszy element)
        for col_name, (func, prefix) in strategies.items():
            self.df[col_name] = self.df["Prompt"].apply(func)
            self.columns_with_prompts.append(col_name)
        for col_name in self.columns_with_prompts:
            self.df[col_name] = self.df[col_name].apply(lambda x: f"{x} \nZacznij odpowiedź od stwierdzenia: '{self.positive_response}' albo '{self.negative_response}', jeżeli coś nie jest w 100% prawdziwe, to odpowiedź uznaj za fałszywe.")

    async def prompts(self):
        total_columns = len(self.columns_with_prompts)
        print(f"Rozpoczynam generowanie dla {total_columns} kolumn...")
        
        requests_per_minute = self.rate_limit
        delay_seconds = 60.0 / requests_per_minute

        total_ok = 0
        total_err = 0
        
        for col_index, col_name in enumerate(self.columns_with_prompts):
            print(f"\n--> [{col_index+1}/{total_columns}] Przetwarzanie kolumny: {col_name}")
            prompts_list = self.df[col_name].tolist()
            column_results = []
            
            pbar = tqdm(prompts_list, desc=f"Generowanie '{col_name}'", unit="prompt")
            for prompt in pbar:
                loop_start_time = time.time()

                try:
                    response = await self.client.aio.models.generate_content(
                        model='gemma-3-4b-it',
                        contents=prompt,
                        config=types.GenerateContentConfig(temperature=0.5)
                    )
                    cleaned_res = response.text.strip()
                    column_results.append(cleaned_res)
                    total_ok += 1
                except Exception as e:
                    error_msg = f"Error: {str(e)}"
                    column_results.append(error_msg)
                    total_err += 1
                
                pbar.set_postfix({'OK': total_ok, 'ERR': total_err})

                elapsed = time.time() - loop_start_time
                wait_time = max(0, delay_seconds - elapsed)
                
                if wait_time > 0:
                    await asyncio.sleep(wait_time)
            
            self.responses[col_name] = column_results
            
        print("\nZakończono generowanie wszystkich odpowiedzi.")
        print("-" * 30)
        print(f"Sukcesy: {total_ok}")
        print(f"Błędy:   {total_err}")
        print("-" * 30)
        return self.responses
    
    def parsowanie(self):
        if self.responses.empty:
            print("Brak odpowiedzi do sparsowania.")
            return self.responses_parsed

        print(f"Rozpoczynam parsowanie {len(self.responses.columns)} kolumn...")

        self.responses_parsed = pd.DataFrame(index=self.responses.index, columns=self.responses.columns)
        
        pos_clean = self.positive_response.replace("'", "").replace('"', "")
        neg_clean = self.negative_response.replace("'", "").replace('"', "")

        def parse_single_response(text):
            if not isinstance(text, str):
                return None
            
            if (self.positive_response in text) or (pos_clean in text):
                return 1
            else:
                return 0

        for col_name in self.responses.columns:
            print(f"--> Parsowanie kolumny: {col_name}")
            self.responses_parsed[col_name] = self.responses[col_name].apply(parse_single_response)
        
        if 'Flag' in self.df.columns:
            self.responses_parsed['Flag'] = self.df['Flag']
            print("--> Przeniesiono kolumnę 'Flag' do wyników.")
        else:
            print("--> UWAGA: Nie znaleziono kolumny 'Flag' w self.df!")

        parsed_count = self.responses_parsed.notna().sum().sum()
        if 'Flag' in self.responses_parsed.columns:
            parsed_count -= self.responses_parsed['Flag'].count() # Odejmujemy Flag, żeby liczyć tylko sparsowane AI
            
        print("-" * 30)
        print(f"Parsowanie zakończone. Skutecznie przetworzono {parsed_count} rekordów.")
        
        return self.responses_parsed
    
    def save_results(self, folder_name="saved_responses"):
        if not os.path.exists(folder_name):
            try:
                os.makedirs(folder_name)
                print(f"Utworzono folder: {folder_name}")
            except OSError as e:
                print(f"Błąd przy tworzeniu folderu: {e}")
                return


        raw_filename = f"raw_responses.csv"
        raw_path = os.path.join(folder_name, raw_filename)
        
        parsed_filename = f"parsed_responses.csv"
        parsed_path = os.path.join(folder_name, parsed_filename)

        try:
            if not self.responses.empty:
                self.responses.to_csv(raw_path, index=True, encoding='utf-8-sig', )
                print(f"Zapisano surowe odpowiedzi: {raw_path}")
            else:
                print("Pominięto zapis surowych odpowiedzi (DataFrame jest pusty).")

            if not self.responses_parsed.empty:
                self.responses_parsed.to_csv(parsed_path, index=True, encoding='utf-8-sig')
                print(f"Zapisano sparsowane wyniki: {parsed_path}")
            else:
                print("Pominięto zapis sparsowanych wyników (DataFrame jest pusty).")
            
        except Exception as e:
            print(f"Problem: {e}")

    def load_results(self, folder_name="saved_responses"):
        raw_path = os.path.join(folder_name, "raw_responses.csv")
        parsed_path = os.path.join(folder_name, "parsed_responses.csv")

        try:
            if os.path.exists(raw_path):
                self.responses = pd.read_csv(raw_path, index_col=0)
                print(f"Wczytano surowe odpowiedzi z: {raw_path}")
            else:
                print(f"Brak pliku surowych odpowiedzi: {raw_path}")

            if os.path.exists(parsed_path):
                self.responses_parsed = pd.read_csv(parsed_path, index_col=0)
                print(f"Wczytano sparsowane wyniki z: {parsed_path}")
            else:
                print(f"Brak pliku sparsowanych wyników: {parsed_path}")

        except Exception as e:
            print(f"Problem przy wczytywaniu wyników: {e}")

    def compute_accuracy_and_variance(self, folder_name="saved_responses"):
        """Oblicz accuracy per strategia (z pominięciem Flag) oraz wariancję odpowiedzi."""
        if self.responses_parsed.empty:
            print("Brak sparsowanych wyników do analizy.")
            return None, None

        if 'Flag' not in self.responses_parsed.columns:
            print("Brak kolumny 'Flag' w responses_parsed.")
            return None, None

        y_true = self.responses_parsed['Flag'].values
        metrics = {'Strategy': [], 'Accuracy': []}

        for col in self.responses_parsed.columns:
            if col == 'Flag':
                continue  # Kolumna referencyjna, zawsze poprawna

            y_pred = self.responses_parsed[col].values
            mask = ~pd.isna(y_pred)
            if mask.sum() == 0:
                continue
            acc = (y_true[mask] == y_pred[mask]).mean()
            metrics['Strategy'].append(col)
            metrics['Accuracy'].append(acc)

        metrics_df = pd.DataFrame(metrics).sort_values('Accuracy', ascending=False).reset_index(drop=True)

        variability = []
        pred_cols = [c for c in self.responses_parsed.columns if c != 'Flag']
        for idx in self.responses_parsed.index:
            row = self.responses_parsed.loc[idx, pred_cols].dropna()
            if len(row) > 1:
                variability.append({
                    'Idx': idx,
                    'Prompt': str(self.df.loc[idx, 'Prompt'])[:80] + '...',
                    'True': y_true[idx] if idx < len(y_true) else None,
                    'Std': row.std()
                })

        var_df = pd.DataFrame(variability).sort_values('Std', ascending=False).reset_index(drop=True)

        if folder_name:
            os.makedirs(folder_name, exist_ok=True)
            metrics_path = os.path.join(folder_name, "metrics.csv")
            var_path = os.path.join(folder_name, "controversial.csv")
            metrics_df.to_csv(metrics_path, index=False, encoding='utf-8-sig')
            var_df.to_csv(var_path, index=False, encoding='utf-8-sig')
            print(f"Zapisano metryki (bez kolumny Flag): {metrics_path}")
            print(f"Zapisano przypadki kontrowersyjne: {var_path}")

        self.metrics_df = metrics_df
        self.var_df = var_df
        return metrics_df, var_df


In [37]:
# --- Wywołanie (Przykład) ---
data_model = DataModel(df, client, rate_limit=29) # max może być 30
data_model.make_prompts()
data_model()


Typ kolumny Prompt: <class 'pandas.core.series.Series'>
Wymiary danych: (60, 13)
Dostępne kolumny: ['Prompt', 'Flag', 'Negative_prompt', 'Positive_prompt', 'Positive_Extra_role', 'Negative_Extra_role', 'Uncertainty_prompt', 'Scrambled_prompt', 'Chain_of_thoughts', 'Sceptical_role', 'High_stakes', 'Tipping', 'Random_mistake']
Kolumny z promptami do przetworzenia: ['Prompt', 'Negative_prompt', 'Positive_prompt', 'Positive_Extra_role', 'Negative_Extra_role', 'Uncertainty_prompt', 'Scrambled_prompt', 'Chain_of_thoughts', 'Sceptical_role', 'High_stakes', 'Tipping', 'Random_mistake']


In [38]:
if CZY_GENEROWAC:
    responses_df = await data_model.prompts()
else:
    data_model.load_results()
    responses_df = data_model.responses

Wczytano surowe odpowiedzi z: saved_responses\raw_responses.csv
Wczytano sparsowane wyniki z: saved_responses\parsed_responses.csv


In [39]:
responses_df.head()

,Prompt,Negative_prompt,Positive_prompt,Positive_Extra_role,Negative_Extra_role,Uncertainty_prompt,Scrambled_prompt,Chain_of_thoughts,Sceptical_role,High_stakes,Tipping,Random_mistake
0,STATUS_ZDANIA:PRAWDA\n\n**Dowód:**\n\nNiech $...,"STATUS_ZDANIA:PRAWDA\n\nZdanie ""Przecięcie dow...",STATUS_ZDANIA:PRAWDA\n\nTo zdanie jest poprawn...,STATUS_ZDANIA:FAŁSZ\n\n**Wyjaśnienie:**\n\nTo ...,STATUS_ZDANIA:FAŁSZ\n\n**Wyjaśnienie:**\n\nTo ...,STATUS_ZDANIA:PRAWDA,STATUS_ZDANIA:FAŁSZ,"STATUS_ZDANIA:PRAWDA\n\n**Analiza zdania: ""Prz...",STATUS_ZDANIA:FAŁSZ\n\nZgadzam się z twierdzen...,STATUS_ZDANIA:FAŁSZ\n\n**Wyjaśnienie:**\n\nTo ...,STATUS_ZDANIA:PRAWDA,STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nZacznij...
1,STATUS_ZDANIA:PRAWDA\n\nSkończona addytywność ...,STATUS_ZDANIA:PRAWDA\n\nTwoje zdanie jest popr...,STATUS_ZDANIA:PRAWDA\n\nTwoje zdanie jest popr...,STATUS_ZDANIA:FAŁSZ\n\nSkończona addytywność p...,STATUS_ZDANIA:PRAWDA\n\n**Wyjaśnienie:**\n\nTo...,STATUS_ZDANIA:PRAWDA,STATUS_ZDANIA:FAŁSZ\n\nTekst jest kompletnie b...,"STATUS_ZDANIA:PRAWDA\n\n**Analiza zdania ""Skoń...","STATUS_ZDANIA:FAŁSZ\n\nTo stwierdzenie jest, d...",STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nSkończo...,STATUS_ZDANIA:PRAWDA,STATUS_ZDANIA:FAŁSZ\n\n**Wyjaśnienie:**\n\nSko...
2,STATUS_ZDANIA:PRAWDA\n\nTo stwierdzenie jest z...,"STATUS_ZDANIA:FAŁSZ\n\nZdanie ""Jeśli ciąg zdar...",STATUS_ZDANIA:FAŁSZ\n\nZdanie jest nie do końc...,STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nTo stwi...,STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nTo stwi...,STATUS_ZDANIA:PRAWDA\n\n**Wyjaśnienie:**\n\nTo...,STATUS_ZDANIA:FAŁSZ,STATUS_ZDANIA:FAŁSZ\n\n**Analiza zdania krok p...,STATUS_ZDANIA:FAŁSZ\n\nTo stwierdzenie jest ni...,STATUS_ZDANIA:FAŁSZ\n\n**Wyjaśnienie:**\n\nTo ...,STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nZdanie ...,STATUS_ZDANIA:PRAWDA\n\nWyjaśnienie:\n\nTo stw...
3,STATUS_ZDANIA:PRAWDA\n\nZbiór Vitaliego jest p...,STATUS_ZDANIA:PRAWDA\n\nTo zdanie jest prawdzi...,STATUS_ZDANIA:PRAWDA\n\nTo zdanie jest poprawn...,STATUS_ZDANIA:PRAWDA\n\n**Wyjaśnienie:**\n\nZb...,STATUS_ZDANIA:PRAWDA\n\n**Wyjaśnienie:**\n\nZb...,STATUS_ZDANIA:PRAWDA,STATUS_ZDANIA:FAŁSZ,STATUS_ZDANIA:PRAWDA\n\n**Analiza zdania krok ...,"STATUS_ZDANIA:FAŁSZ\n\nZbiór Vitaliego, choć f...",STATUS_ZDANIA:FAŁSZ\n\nZbiór Vitaliego jest pr...,STATUS_ZDANIA:PRAWDA,STATUS_ZDANIA:PRAWDA\n\n**Wyjaśnienie:**\n\nZb...
4,STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nZdarzen...,STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nZdanie ...,STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nZdarzen...,STATUS_ZDANIA:FAŁSZ\n\n**Wyjaśnienie:**\n\nZda...,STATUS_ZDANIA:FAŁSZ\n\n**Wyjaśnienie:**\n\nTo ...,STATUS_ZDANIA:FAŁSZ,STATUS_ZDANIA:FAŁSZ\n\nTekst jest chaotyczny i...,STATUS_ZDANIA:FAŁSZ\n\n**Analiza krok po kroku...,STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nTo stwi...,STATUS_ZDANIA:FAŁSZ\n\n**Wyjaśnienie:**\n\nZda...,STATUS_ZDANIA:FAŁSZ,STATUS_ZDANIA:PRAWDA\n\nWyjaśnienie:\n\nJeśli ...


In [40]:

parsed_df = data_model.parsowanie()

# Dane są już załadowane z pliku parsed_responses_*.csv
parsed_df.head()

Rozpoczynam parsowanie 12 kolumn...
--> Parsowanie kolumny: Prompt
--> Parsowanie kolumny: Negative_prompt
--> Parsowanie kolumny: Positive_prompt
--> Parsowanie kolumny: Positive_Extra_role
--> Parsowanie kolumny: Negative_Extra_role
--> Parsowanie kolumny: Uncertainty_prompt
--> Parsowanie kolumny: Scrambled_prompt
--> Parsowanie kolumny: Chain_of_thoughts
--> Parsowanie kolumny: Sceptical_role
--> Parsowanie kolumny: High_stakes
--> Parsowanie kolumny: Tipping
--> Parsowanie kolumny: Random_mistake
--> Przeniesiono kolumnę 'Flag' do wyników.
------------------------------
Parsowanie zakończone. Skutecznie przetworzono 720 rekordów.


,Prompt,Negative_prompt,Positive_prompt,Positive_Extra_role,Negative_Extra_role,Uncertainty_prompt,Scrambled_prompt,Chain_of_thoughts,Sceptical_role,High_stakes,Tipping,Random_mistake,Flag
0,1,1,1,0,0,1,0,1,0,0,1,0,1
1,1,1,1,0,1,1,0,1,0,0,1,0,0
2,1,0,0,0,0,1,0,0,0,0,0,1,1
3,1,1,1,1,1,1,0,1,0,0,1,1,0
4,0,0,0,0,0,0,0,0,0,0,0,1,0


In [41]:
print(responses_df.head())

                                              Prompt  \
0  STATUS_ZDANIA:PRAWDA\n\n**Dowód:**\n\nNiech  $...   
1  STATUS_ZDANIA:PRAWDA\n\nSkończona addytywność ...   
2  STATUS_ZDANIA:PRAWDA\n\nTo stwierdzenie jest z...   
3  STATUS_ZDANIA:PRAWDA\n\nZbiór Vitaliego jest p...   
4  STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nZdarzen...   

                                     Negative_prompt  \
0  STATUS_ZDANIA:PRAWDA\n\nZdanie "Przecięcie dow...   
1  STATUS_ZDANIA:PRAWDA\n\nTwoje zdanie jest popr...   
2  STATUS_ZDANIA:FAŁSZ\n\nZdanie "Jeśli ciąg zdar...   
3  STATUS_ZDANIA:PRAWDA\n\nTo zdanie jest prawdzi...   
4  STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nZdanie ...   

                                     Positive_prompt  \
0  STATUS_ZDANIA:PRAWDA\n\nTo zdanie jest poprawn...   
1  STATUS_ZDANIA:PRAWDA\n\nTwoje zdanie jest popr...   
2  STATUS_ZDANIA:FAŁSZ\n\nZdanie jest nie do końc...   
3  STATUS_ZDANIA:PRAWDA\n\nTo zdanie jest poprawn...   
4  STATUS_ZDANIA:FAŁSZ\n\nWyjaśnienie:\n\nZdar

In [42]:
print("PRZYKŁADOWE ODPOWIEDZI Z MODELU GEMMA")

for col in responses_df.columns[:3]:
    print(f"\n{'='*80}")
    print(f"KOLUMNA: {col}")
    print(f"{'='*80}")
    for i in range(min(5, len(responses_df))):  # Pierwsze 5 wierszy
        print(f"\n[{i}] {responses_df[col].iloc[i][:200]}...")  # Pierwsze 200 znaków
        print("-"*40)

PRZYKŁADOWE ODPOWIEDZI Z MODELU GEMMA

KOLUMNA: Prompt

[0] STATUS_ZDANIA:PRAWDA

**Dowód:**

Niech  $A_1, A_2, A_3, ...$ będzie nieprzeliczalną rodziną σ-ciał w przestrzeni topologicznej $X$.  Chcemy pokazać, że ich przecięcie, $A = \bigcap_{i=1}^{\infty} A_i...
----------------------------------------

[1] STATUS_ZDANIA:PRAWDA

Skończona addytywność prawdopodobieństwa jest warunkiem silniejszym niż przeliczalna addytywność. Oznacza to, że jeśli przestrzeń prawdopodobieństwa spełnia warunek skończonej ad...
----------------------------------------

[2] STATUS_ZDANIA:PRAWDA

To stwierdzenie jest zasadniczo prawdziwe w kontekście teorii prawdopodobieństwa.  Jeśli mamy do czynienia z ciągiem zdarzeń, które są *wstępujące* (czyli jedno zdarzenie musi na...
----------------------------------------

[3] STATUS_ZDANIA:PRAWDA

Zbiór Vitaliego jest przykładem zbioru, który jest borelowski, ale nie jest mierzalny w sensie Lebesgue'a. Zbiór Vitaliego jest konstruowany na przestrzeni metrycznej n

In [43]:
metrics_df, var_df = data_model.compute_accuracy_and_variance()
metrics_df.to_csv(f'saved_responses/metrics.csv', index=False, encoding='utf-8-sig')
var_df.to_csv(f'saved_responses/controversial.csv', index=False, encoding='utf-8-sig')

print(f"\n[OK] Gotowe!")
print(f"  Metrics saved: metrics.csv")
print(f"  Cases saved:   controversial.csv")

Zapisano metryki (bez kolumny Flag): saved_responses\metrics.csv
Zapisano przypadki kontrowersyjne: saved_responses\controversial.csv

[OK] Gotowe!
  Metrics saved: metrics.csv
  Cases saved:   controversial.csv


In [45]:
# Tworzenie tabeli z wynikami
results_table = metrics_df.copy()
results_table['Prefix'] = results_table['Strategy'].map(lambda x: data_model.strategies.get(x, ''))
results_table = results_table.set_index('Strategy').sort_values('Accuracy', ascending=False)
results_table

,Accuracy,Prefix
Strategy,,
Sceptical_role,0.650000,"Jesteś sceptykiem, który zawsze podważa prawdz..."
Prompt,0.600000,
Chain_of_thoughts,0.600000,Przeanalizuj dokładnie to zdanie krok po kroku...
High_stakes,0.600000,"Oceń prawdziwość, pamiętając, że od twojej odp..."
Random_mistake,0.600000,Oceń prawdziwość:
Tipping,0.600000,Dostaniesz duży napiwek jeśli prawidłowo oceni...
Uncertainty_prompt,0.583333,Oceń prawdziwość tego zdania tylko jeżeli jest...
Negative_Extra_role,0.583333,Jesteś ekspertem w dziedzinie socjologii oraz ...
Negative_prompt,0.550000,"Myślę, że to zdanie zawiera problem, czy mam r..."
